In [2]:
import textwrap
import json
import random
import os

# 定義不同級別的提示
leight_prompt = textwrap.dedent("""
使用判決書填充JSON結構。要求如下:
1. 以賠償給原告的數據填寫。
2. 判決前的賠償金額需要包含：零件、材料、塗裝、烤漆。
返回結果為一行JSON格式字串，無換行或特殊符號。
""")

medium_prompt = textwrap.dedent("""
根據給定的判決文檔填充JSON結構。要求如下:
1. 以賠償給原告的數據填寫。
2. 判決前的賠償金額需包含：零件、材料、工資、鈑金、塗裝、烤漆。
3. 若無結果則留空白，如 {"工資": ""} 
4. 注意每日每月之單位，一個月為30天
5. 每日工作收入改填寫每月工作收入
返回結果為一行JSON格式字串，無換行或特殊符號。
""")

rule_level_list = {
    'light': leight_prompt,
    'medium': medium_prompt,
}

all_data_path = "./finetuning_training_data_golden.jsonl"

# 讀取資料
with open(all_data_path, 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

random.shuffle(data)

split_ratio = 0.8
split_index = int(len(data) * split_ratio)

# 分割資料
train_data = data[:split_index]
remaining_data = data[split_index:]
val_test_split_index = int(len(remaining_data) * 0.5)

val_data = remaining_data[:val_test_split_index]
test_data = remaining_data[val_test_split_index:]

def format_data(prompt, data_item):
    formatted_text = f"<s>[INST]{prompt}[/INST] [CONTENT]{data_item['input']}[/CONTENT] {data_item['output']}</s>"
    return {'text': formatted_text}

for rule_level, prompt in rule_level_list.items():

    # 定義目錄
    dir_path = f"./ccg/{rule_level}"

    # 檢查並創建目錄
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

    train_data_path = os.path.join(dir_path, "train.jsonl")
    val_data_path = os.path.join(dir_path, "valid.jsonl")
    test_data_path = os.path.join(dir_path, "test.jsonl")

    # 在每個資料項目中加入 subject 並格式化
    train_data_with_subject = [format_data(prompt, item) for item in train_data]
    val_data_with_subject = [format_data(prompt, item) for item in val_data]
    test_data_with_subject = [format_data(prompt, item) for item in test_data]

    # 儲存 train 資料
    with open(train_data_path, 'w', encoding='utf-8') as f:
        for item in train_data_with_subject:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

    # 儲存 valid 資料
    with open(val_data_path, 'w', encoding='utf-8') as f:
        for item in val_data_with_subject:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

    # 儲存 test 資料
    with open(test_data_path, 'w', encoding='utf-8') as f:
        for item in test_data_with_subject:
            f.write(json.dumps(item, ensure_ascii=False) + '\n')

    print(f"{rule_level} 資料分割完成！")


light 資料分割完成！
medium 資料分割完成！
